# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import numpy as np

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_data.db')
df = pd.read_sql_table("disaster_Fact", engine)
X = df["message"]
Y = df[df.columns[4:]]

In [3]:
df.shape

(26216, 40)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text: str):
    lower_case = text.lower()
    words = re.sub(r'[^a-zA-Z0-9]', " ", lower_case)
    words = words.split()
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('predictor', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)

In [18]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x14c9130a0>)),
                ('tfidf', TfidfTransformer()),
                ('predictor',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [23]:
from joblib import dump
dump(pipeline, "pipeline.joblib")

['pipeline.joblib']

In [5]:
from joblib import load
model = load("pipeline.joblib")

FileNotFoundError: [Errno 2] No such file or directory: 'pipeline.joblib'

In [6]:
y_pred = model.predict(X_test)

NameError: name 'model' is not defined

In [7]:
y_test.iloc[:, 1].shape, y_pred[:,1].shape

((5244,), (5244,))

In [17]:
idx = 0
print(classification_report(y_pred[:,idx], y_test.iloc[:, idx]))

              precision    recall  f1-score   support

           0       0.82      0.97      0.89      1042
           1       0.99      0.95      0.97      4169
           2       0.80      0.97      0.88        33

    accuracy                           0.95      5244
   macro avg       0.87      0.96      0.91      5244
weighted avg       0.96      0.95      0.95      5244



In [28]:
weighted_precision = []
avg_precision = []
weighted_recall = []
avg_recall = []
weighted_f1 = []
avg_f1 = []

for i, key in enumerate(y_test):
    report = classification_report(y_pred[:, i], y_test.iloc[:, i], output_dict=True)
    avg_precision.append(report['macro avg']['precision'])
    avg_recall.append(report['macro avg']['recall'])
    avg_f1.append(report['macro avg']['f1-score'])
    weighted_precision.append(report['weighted avg']['precision'])
    weighted_recall.append(report['weighted avg']['recall'])
    weighted_f1.append(report['weighted avg']['f1-score'])

In [32]:
np.mean(np.asarray(weighted_precision)), np.mean(np.asarray(avg_precision))

(0.9903380790739453, 0.9141741559124321)

In [33]:
np.mean(np.asarray(weighted_recall)), np.mean(np.asarray(avg_recall))

(0.9888655818289686, 0.9905983939066285)

In [34]:
np.mean(np.asarray(weighted_f1)), np.mean(np.asarray(avg_f1))

(0.9892420093314571, 0.9481346991230508)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    "predictor__estimator__criterion": ["gini", "entropy"],
    "predictor__estimator__max_features": ["sqrt", "log2"]
}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [1]:
from sklearn.ensemble import GradientBoostingClassifier

### 9. Export your model as a pickle file

In [ ]:
from joblib import dump
estimator = cv.best_estimator_
dump(estimator, "best-model.joblib")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [13]:
from joblib import load
from sklearn.ensemble import RandomForestClassifier
model = load("web_app/models/best-model.joblib")

/opt/homebrew/Caskroom/miniforge/base/envs/etl/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.19.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/etl/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


ModuleNotFoundError: No module named 'sklearn.ensemble.forest'